# This is a sandbox for setting up a run of the simulation on `TNC` only. 

* It can be run on a small subsample pandas DataFrame


In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../../data'
mode = 'ON_DEMAND_AUTO'
simulation_id = 'dev'
test = True

#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../../data/EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))
    # df = df.loc[df['mode'] == mode]
    df = df.sample(frac=0.001, random_state=1)
    # df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
    simulation_id = 'dev'
    print(len(df))

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]

51727


In [3]:
df['mode'].value_counts()

mode
PRIVATE_AUTO         27033
CARPOOL              16445
WALKING               4966
COMMERCIAL            1455
OTHER_TRAVEL_MODE      664
ON_DEMAND_AUTO         457
PUBLIC_TRANSIT         401
BIKING                 306
Name: count, dtype: int64

In [4]:
df_heavy = df.loc[df['vehicle_type'] == 'HEAVY_COMMERCIAL']
df_medium = df.loc[df['vehicle_type'] == 'HEAVY_COMMERCIAL']

df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
print(len(df))
#df = df.compute()

457


### figure out how to determine P(charge) and how many trips that will take. 

In [5]:
# this is the instance where we see how many trips it took to get to max range

range_mi = 200
tot_distance = 0
trip_counter = 0
trip_list=[]

for i in range(len(df)):
    trip = df.iloc[i,]
    tot_distance += trip['distance_miles']
    trip_counter += 1
    if tot_distance > range_mi:
        trip_list.append(trip_counter)
        tot_distance = 0
        trip_counter = 0

p_charge=np.mean(trip_list)/range_mi
p_charge

0.12416666666666666

run the simulation on small dataset to test. 

In [6]:
# this takes ~1min for 10,000 trips
# out = sim.simulate_person_load(df, existing_load, 'base', managed=False)

embarrassingly parallel

In [7]:
number_of_chunks = 10
df_list = np.array_split(df, number_of_chunks)

In [8]:
# run the simulation in parallel
# df must be a pandas dataframe

charge_sims = joblib.Parallel(verbose=10, n_jobs=-1)(joblib.delayed(sim.simulate_person_load)(trips_df=df_i, 
        existing_load=existing_load,
        simulation_id=simulation_id,
        managed=False) 
     for df_i in df_list)

# sim.simulate_person_load(trips_df=df, 
#         existing_load=existing_load,
#         simulation_id=simulation_id,
#         managed=False) 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    4.7s remaining:   10.9s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    4.7s remaining:    4.7s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    4.7s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.7s finished


### Restack the DataFrames

In [9]:
# restack the dataframes
charges_list = [x['charges'] for x in charge_sims]
loads_list = [x['loads'] for x in charge_sims]

Pandas significantly faster since we can fit these in memory. 

In [11]:
# restack the dataframes
charges_df = pd.concat(charges_list)
loads_df = pd.concat(loads_list) # huge ~200 million rows

In [21]:
loads_df.loc[loads_df.person_id=='11733814650649905888']

,person_id,load_segment_id,charge_id,window_start_time,window_end_time,load_kW
0,11733814650649905888,15287084765420456353_dev_0,15287084765420456353_dev,0 days 00:00:00,0 days 01:00:00,0.000000
1,11733814650649905888,15287084765420456353_dev_1,15287084765420456353_dev,0 days 01:00:00,0 days 02:00:00,0.000000
2,11733814650649905888,15287084765420456353_dev_2,15287084765420456353_dev,0 days 02:00:00,0 days 03:00:00,0.000000
3,11733814650649905888,15287084765420456353_dev_3,15287084765420456353_dev,0 days 03:00:00,0 days 04:00:00,0.000000
4,11733814650649905888,15287084765420456353_dev_4,15287084765420456353_dev,0 days 04:00:00,0 days 05:00:00,0.000000
5,11733814650649905888,15287084765420456353_dev_5,15287084765420456353_dev,0 days 05:00:00,0 days 06:00:00,0.000000
6,11733814650649905888,15287084765420456353_dev_6,15287084765420456353_dev,0 days 06:00:00,0 days 07:00:00,0.000000
7,11733814650649905888,15287084765420456353_dev_7,15287084765420456353_dev,0 days 07:00:00,0 days 08:00:00,0.000000
8,11733814650649905888,15287084765420456353_dev_8,15287084765420456353_dev,0 days 08:00:00,0 days 09:00:00,0.000000
9,11733814650649905888,15287084765420456353_dev_9,15287084765420456353_dev,0 days 09:00:00,0 days 10:00:00,0.000000


,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
43,15287084765420456353,11733814650649905888,ON_DEMAND_AUTO,RECREATION,HOME,0 days 17:35:38,0 days 17:59:56,8.479734,nan,530330260013,47.500313,-122.231509,530330089001,47.596195,-122.27968,residential,single_family,residential,single_family,mixed_use,non_retail_attraction,open_space,open_space,47.501172,-122.23689,47.60117,-122.285084,saturday,7772624807417728476,530330260013,nan,nan,53033026001,nan,nan,65_plus,80.0,F,white,not_hispanic_or_latino,100000_plus,131385.0,notinlf,high_school,not_attending_school,not_working,head_of_household,0,nan,not_working,renter,same_house,1_person,125000_plus,131385.0,living_alone,2,single_family,core,english,47.501172,-122.23689,NaN,NaN,NaN,NaN,under_16_or_notinlf,PUBLIC


In [18]:
trips=df[df['person_id']=='11733814650649905888'].copy()
trips['stop_duration'] = 0
    # For each row in the trips table, calculate the stop duration
for i in range(0, len(trips)-1):
    trips.iloc[i, trips.columns.get_loc('stop_duration')] =\
        trips.iloc[i+1, trips.columns.get_loc('start_time')] -\
        trips.iloc[i, trips.columns.get_loc('end_time')]

In [19]:
trips

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,stop_duration
43,15287084765420456353,11733814650649905888,ON_DEMAND_AUTO,RECREATION,HOME,0 days 17:35:38,0 days 17:59:56,8.479734,nan,530330260013,47.500313,-122.231509,530330089001,47.596195,-122.27968,residential,single_family,residential,single_family,mixed_use,non_retail_attraction,open_space,open_space,47.501172,-122.23689,47.60117,-122.285084,saturday,7772624807417728476,530330260013,nan,nan,53033026001,nan,nan,65_plus,80.0,F,white,not_hispanic_or_latino,100000_plus,131385.0,notinlf,high_school,not_attending_school,not_working,head_of_household,0,nan,not_working,renter,same_house,1_person,125000_plus,131385.0,living_alone,2,single_family,core,english,47.501172,-122.23689,NaN,NaN,NaN,NaN,under_16_or_notinlf,PUBLIC,0


In [ ]:
charges_df.to_parquet(os.path.join(datadir, f'charges_{mode}_{simulation_id}.parquet'), overwrite=True)
loads_df.to_parquet(os.path.join(datadir, f'loads_{mode}_{simulation_id}.parquet'), overwrite=True)